In [2]:
# Import the required libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm 
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets.samples_generator import make_blobs

In [3]:
#Load in South American Real Estate "For Sale" Listing Data Set and drop Unamed index columns
# Create data set 'df'
df = pd.read_csv('Imputed_Dataset.csv')
df.drop(df.columns[df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
df.info()
df.dtypes
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463724 entries, 0 to 463723
Data columns (total 20 columns):
lat                409210 non-null float64
lon                409210 non-null float64
rooms              463724 non-null float64
bedrooms           463724 non-null float64
bathrooms          463724 non-null float64
surface_total      463724 non-null float64
surface_covered    463724 non-null float64
price              463723 non-null float64
log_price          463724 non-null float64
id                 463723 non-null object
start_date         463723 non-null object
end_date           463723 non-null object
created_on         463723 non-null object
property_type      463723 non-null object
l1                 463723 non-null object
l2                 463723 non-null object
l3                 441216 non-null object
title              463723 non-null object
description        463717 non-null object
price_class        463723 non-null object
dtypes: float64(9), object(11)
memory us

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,log_price,id,start_date,end_date,created_on,property_type,l1,l2,l3,title,description,price_class
0,NaN,NaN,4.0,3.0,2.0,198.0,150.0,385000.0,12.860999,EWeY8jVhb8ielLeKtfibVw==,2019-09-15,2020-01-20,2019-09-15,Casa,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,"Venta. Casa en una planta, 4 amb. con galería ...",MAM.(2) Hermosa propiedad en Barrio San Franci...,High
1,NaN,NaN,4.0,3.0,2.0,198.0,150.0,385000.0,12.860999,IrEeG8ewIcfLVGSb14kH1w==,2019-09-15,2020-01-20,2019-09-15,Casa,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Venta. Casa en una planta.Lote interno. Barrio...,MAM. Hermosa propiedad en Barrio San Francisco...,High
2,-27.371199,-55.898454,7.0,3.0,2.0,173.0,173.0,195000.0,12.180755,hPuiyAjuBI92uEQdFoY4Fw==,2019-09-15,1970-01-01,2019-09-15,Casa,Argentina,Misiones,Posadas,Casa - Posadas,Inmueble centrico ideal para Local comercial ...,Average
3,NaN,NaN,3.0,3.0,1.0,49.0,40.0,85000.0,11.350407,9vIWv494LqC6crAkfpLeDg==,2019-09-15,2020-03-03,2019-09-15,Casa,Argentina,Santa Fe,Rosario,Cabaña en la Isla sobre riacho Los Marinos - V...,Lote de 20 metros de frente por 130 metros de ...,Low
4,-32.951146,-60.571979,3.0,3.0,1.0,49.0,40.0,85000.0,11.350407,aydrtgoZ/frzXDcV20McuA==,2019-09-15,2020-03-03,2019-09-15,Casa,Argentina,Entre Ríos,Victoria,Cabaña en la Isla sobre riacho Los Marinos - V...,Lote de 20 metros de frente por 130 metros de ...,Low


In [4]:
df.property_type.unique()

array(['Casa', 'Departamento', 'Local comercial', 'PH', 'Lote', 'Otro',
       'Garaje', 'Oficina', 'Depósito', 'Casa de campo', 'Finca',
       'Parqueadero', nan], dtype=object)

In [5]:
df.l1.unique()

array(['Argentina', 'Uruguay', 'Colombia', 'Ecuador', 'Perú', nan],
      dtype=object)

In [6]:
# In dataset df, drop specific columns that is not needed in SVM
# Create data set 'df1' without remaining attributes
df1 = df.drop(['lat', 'lon', 'start_date','end_date', 'created_on','l2','l3', 'title','description','id','price'], axis=1)
df1.head(5)

,rooms,bedrooms,bathrooms,surface_total,surface_covered,log_price,property_type,l1,price_class
0,4.0,3.0,2.0,198.0,150.0,12.860999,Casa,Argentina,High
1,4.0,3.0,2.0,198.0,150.0,12.860999,Casa,Argentina,High
2,7.0,3.0,2.0,173.0,173.0,12.180755,Casa,Argentina,Average
3,3.0,3.0,1.0,49.0,40.0,11.350407,Casa,Argentina,Low
4,3.0,3.0,1.0,49.0,40.0,11.350407,Casa,Argentina,Low


In [7]:
df1[['price_class','property_type']].describe().transpose()

,count,unique,top,freq
price_class,463723,3,High,158761
property_type,463723,12,Departamento,246400


In [8]:
#Defining variable data types
string_vars = ['price_class','property_type','l1']
df1[string_vars] = df1[string_vars].astype(str)


In [9]:
# label encoding property_type, l1, and price_class
from sklearn.preprocessing import LabelEncoder 

le = LabelEncoder() 
df1['property_type']= le.fit_transform(df1['property_type']) 
df1['l1']= le.fit_transform(df1['l1']) 
df1['price_class'] = le.fit_transform(df1['price_class'])
df1.head(5)

,rooms,bedrooms,bathrooms,surface_total,surface_covered,log_price,property_type,l1,price_class
0,4.0,3.0,2.0,198.0,150.0,12.860999,0,0,1
1,4.0,3.0,2.0,198.0,150.0,12.860999,0,0,1
2,7.0,3.0,2.0,173.0,173.0,12.180755,0,0,0
3,3.0,3.0,1.0,49.0,40.0,11.350407,0,0,2
4,3.0,3.0,1.0,49.0,40.0,11.350407,0,0,2


In [19]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 
  
# creating one hot encoder object by default 
# entire data passed is one hot encoded 
onehotencoder = OneHotEncoder() 
  
df1 = np.array(columnTransformer.fit_transform(df1), dtype = np.str)

NameError: name 'columnTransformer' is not defined

In [62]:
X = df1.drop('log_price', axis=1)
y = df1['log_price']
print(X.shape)
print (y.shape)

(463724, 8)
(463724,)


In [ ]:
cv = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 40)
for train_index, test_index in cv.split(X, y):
  print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]


In [ ]:
from sklearn.preprocessing import StandardScaler
regularize_const = 0.1
iterations = 3
svm_sgd = SGDClassifier(alpha = regularize_const, fit_intercept=True, l1_ratio = 0.0, learning_rate = 'optimal',
                        loss='hinge', n_iter_no_change=iterations, n_jobs = -1, penalty = 'l2')
scl = StandardScaler()
for train_index,test_index in cv.split(X,y):
    svm_sgd.fit(scl.fit_transform(X[train_index]),y[train_index])
    yhat = svm_sgd.predict(scl.transform(X[test_index]))

In [ ]:
cm = mt.confusion_matrix(y[test_idx],yhat)
acc = mt.accuracy_score(y[test_idx],yhat)

In [ ]:
print ('SVM: ', acc)

In [ ]:
print (cm)